In [2]:
! pip install pymorphy3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.4/55.4 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 75.5 MB/s eta 0:00:00:00:010:01


In [3]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
from tqdm import tqdm 

from gensim.models.word2vec import Word2Vec
from sklearn.metrics import accuracy_score
import re
import pymorphy3

parser = pymorphy3.MorphAnalyzer()

tqdm.pandas()

In [4]:
from gensim.models.word2vec import Word2Vec
model  = Word2Vec.load('/kaggle/input/models/cosmetic_word2vec.model')

In [ ]:
model.wv

# Cosmetics

In [380]:
import os

def get_data(folder, train, val, target):
    train = pd.read_csv(os.path.join("/kaggle/input/", folder, train), sep="\t")
    val = pd.read_csv(os.path.join("/kaggle/input/", folder, val), sep="\t")
    target = pd.read_csv(os.path.join("/kaggle/input/", folder, target), sep="\t") \
        .drop_duplicates() \
        .reset_index(drop=True)
    target.columns=['receipt_id', 'target']
    target.target = target.target.astype(str)
    
    return train, val, target


def get_noun(words: list[str]) -> str:
    result = []
    for word in words:
        parsings = [word for parsing in parser.parse(word)[:3] if parsing.tag.POS == 'NOUN']
        if len(parsings)>0:
            result.append(word)
    
    if len(result)==0:
        return words[0]
    return result[0]


def preparation_data(dataset):
    dataset['processed_name'] = dataset['name']\
    .str.lower()\
    .apply(lambda x: re.sub(r'\s+', ' ', re.sub(r'[^a-zа-я -]', '', x)).strip())

    dataset['category_first_word'] = dataset['processed_name'].str.split(' ')\
                                                .progress_apply(lambda x: x[0])

    dataset['category_noun'] = dataset['processed_name'].str.split(' ')\
                                                .progress_apply(lambda x: get_noun(x[:3]))
    
    dataset['item_id'] = dataset['item_id'].astype(str)
    
    return dataset


def get_products_short_name_map(train, val):
    products_name = pd.concat([train[['item_id', 'category_noun']], val[['item_id', 'category_noun']]]).drop_duplicates()
    products_name.columns = ['target', 'category_noun'] 
    products_name.target = products_name.target.astype(str)
    
    return products_name


In [381]:
folder = "dataset"
train = "cosmetic_train.tsv"
val = "cosmetic_val.tsv"
target = "cosmetic_val_target.tsv"

train, val, target = get_data(folder, train, val, target)

## name extraction

In [382]:
train = preparation_data(train)
val = preparation_data(val)

100%|██████████| 51650/51650 [00:35<00:00, 1474.22it/s]


In [383]:
train['category_noun'].value_counts()[0:50]

category_noun
скраб               40971
крем                12007
обертывание         11680
масло               11513
крем-баттер         11504
гель                10719
коробка             10406
маска                9133
бальзам              8630
сыворотка            6360
соль                 5843
лосьон               5795
спрей                5109
шампунь              4987
пакет                3407
крем-суфле           3286
штаны                3174
кондиционер          3060
щетка                2862
сумка-шоппер         2659
тоник                2350
пенка                2221
эмульсия             2218
антицеллюлитная      2105
крем-корсет          2014
маска-сорбет         1968
крем-гель            1967
мусс                 1881
маска-лифтинг        1879
плампер              1765
помада               1748
флюид                1662
карандаш             1646
сс-крем              1251
массажер             1244
криомасло            1172
молочко              1132
сыворотка-сияние     112

In [384]:
val['category_noun'].value_counts()[0:50]

category_noun
скраб               9236
крем                2801
обертывание         2668
коробка             2655
масло               2590
крем-баттер         2578
гель                2543
маска               2154
бальзам             1957
сыворотка           1472
соль                1405
лосьон              1392
шампунь             1164
спрей               1138
пакет                788
штаны                784
крем-суфле           730
сумка-шоппер         699
кондиционер          686
щетка                681
тоник                533
эмульсия             523
антицеллюлитная      506
пенка                501
крем-корсет          498
крем-гель            455
маска-сорбет         447
маска-лифтинг        424
мусс                 422
плампер              393
карандаш             366
флюид                366
помада               355
сс-крем              282
криомасло            278
сыворотка-сияние     274
массажер             267
молочко              254
гель-пена            243
тонер      

## word2vec

In [385]:
products_name = get_products_short_name_map(train, val)
train_grouped = train.groupby('receipt_id')['category_noun'].apply(list).reset_index()
val_grouped = val.groupby('receipt_id')['category_noun'].apply(list).reset_index()
val_grouped = val_grouped.merge(target, on=["receipt_id"], how="left")\
                         .merge(products_name, on=["target"], how="left")

In [393]:
products_name.rename(columns = {'target':'item_id'}).to_csv('cosmetic_item_id_categ_map.csv', sep=';', index=None)

In [386]:
model = Word2Vec(
    train_grouped.category_noun.tolist(), vector_size=64, 
    sg=1, epochs=50, negative=10, min_count=1, 
    window=3, seed=42, workers=8
)

X = model.wv.key_to_index.keys()
len(X)

86

In [394]:
model.save("cosmetic_word2vec.model")

In [387]:
import numpy as np


def similar_products(v, products, n = 5):
    try:
        ms = model.wv.similar_by_vector(v, topn= n)
    
        new_ms = []
        for j in ms:
#             if j[0] not in products:
            pair = (j[0], j[1])
            new_ms.append(pair)

        return new_ms    
    except Exception as ex:
        return []


def aggregate_vectors(products):
    product_vec = []
    for i in products:
        try:
            product_vec.append(model.wv[i])
        except KeyError:
            continue
        
    return np.mean(product_vec, axis=0)

In [388]:
val_grouped

,receipt_id,category_noun_x,target,category_noun_y
0,9131497060,[скраб],200336,обертывание
1,9132194046,"[крем-баттер, саше]",200296,скраб
2,9132222056,"[спрей, бальзам]",200388,пакет
3,9132282916,"[маска-лифтинг, скраб, масло, коробка, сумка-ш...",200232,крем-суфле
4,9132985649,[крем],200043,бальзам
...,...,...,...,...
22756,16210761455,[крем-гель],200276,флюид
22757,16210803563,[крем-баттер],200668,эмульсия
22758,16210825989,"[крем-баттер, крем-баттер, крем-баттер]",200490,скраб
22759,16210942293,"[масло, бальзам]",200478,скраб


In [389]:
val_grouped['prediction'] = val_grouped['category_noun_x'].apply(lambda x: similar_products(aggregate_vectors(x), x))
val_grouped['predicted'] = val_grouped['prediction'].apply(lambda x: [xi[0] for xi in x])
val_grouped['accuracy_1'] = val_grouped[['category_noun_y', 'predicted']].apply(lambda x: 1 if len(x[1])>0 and x[0]==x[1][0] else 0, axis=1)
val_grouped['accuracy_2'] = val_grouped[['category_noun_y', 'predicted']].apply(lambda x: 1 if  len(x[1])>0 and x[0] in x[1] else 0, axis=1)

In [390]:
val_grouped['accuracy_2'].value_counts()

accuracy_2
0    15591
1     7170
Name: count, dtype: int64

In [391]:
val_grouped['accuracy_1'].value_counts()

accuracy_1
0    20982
1     1779
Name: count, dtype: int64

In [392]:
print("hit@5 valid: %.2f" % (val_grouped["accuracy_2"].mean() * 100))
print("accuracy valid: %.2f" % (val_grouped["accuracy_1"].mean() * 100))

hit@5 valid: 31.50
accuracy valid: 7.82


# Supermarkets

In [340]:
folder = "dataset"
train = "supermarket_train.tsv"
val = "supermarket_val.tsv"
target = "supermarket_val_target.tsv"

train, val, target = get_data(folder, train, val, target)

## name extraction 

In [341]:
train = preparation_data(train)
val = preparation_data(val)

100%|██████████| 172658/172658 [02:16<00:00, 1263.75it/s]


In [342]:
train['category_noun'].value_counts()[0:50]

category_noun
хлеб            55236
батон           37689
пиво            25055
молоко          24938
напиток         23575
пакет           22432
водка           19649
к-са            18645
морожфабрика    14934
сиг-ты          13357
вода            13071
печенье         11455
корм            10168
пивной           8813
сметана          8202
масло            8123
майонез          7980
сельдь           7304
яйцо             6775
крупа            6263
песок            5337
помидоры         5228
продукт          5226
кефир            5002
вино             4973
бумага           4946
салат            4816
стиль            4811
кофе             4686
чипсы            4490
морожюмо         4480
огурцы           4474
пирожное         4273
пельмени         4272
сыр              3996
картофель        3956
семечки          3911
ватрушка         3909
чай              3853
пицца            3806
сосиски          3794
продукция        3676
н-р              3587
котлеты          3489
пряники          3

In [343]:
val['category_noun'].value_counts()[0:50]

category_noun
хлеб            13835
батон            9547
молоко           6303
пиво             5920
пакет            5855
напиток          5776
к-са             4984
водка            4183
вода             3115
морожфабрика     3070
печенье          2913
корм             2657
сиг-ты           2269
сметана          2184
масло            2132
пивной           2118
майонез          2050
яйцо             1904
сельдь           1785
крупа            1618
помидоры         1465
песок            1442
кефир            1334
бумага           1267
продукт          1238
кофе             1220
огурцы           1210
пельмени         1120
сыр              1096
чипсы            1089
пирожное         1074
ватрушка         1044
салат            1025
чай              1011
морожюмо         1009
картофель        1005
вино              969
сосиски           957
пряники           954
семечки           949
ш-д               938
н-р               897
пицца             873
лук               864
котлеты           

## word2vec

In [362]:
products_name = get_products_short_name_map(train, val)
train_grouped = train.groupby('receipt_id')['category_noun'].apply(list).reset_index()
val_grouped = val.groupby('receipt_id')['category_noun'].apply(list).reset_index()
val_grouped = val_grouped.merge(target, on=["receipt_id"], how="left")\
                         .merge(products_name, on=["target"], how="left")

In [377]:
products_name.rename(columns = {'target':'item_id'}).to_csv('super_item_id_categ_map.csv', sep=';', index=None)

In [364]:
val_grouped

,receipt_id,category_noun_x,target,category_noun_y
0,9127158637,"[сельдь, майонез]",108405,н-р
1,9127241099,"[крупа, сметана, молоко, хлеб, печенье, мука, ...",103481,к-са
2,9127933997,[арбуз],111149,пикша
3,9128240801,"[макиздмф, ватрушка]",105015,кефир
4,9128361592,"[батон, сиг-ты, хлеб, кипятильник, пюре, сосис...",100826,вермишель
...,...,...,...,...
53221,16209384039,[сиг-ты],117114,энергнапиток
53222,16209421467,"[водка, ромамашка]",111051,пиво
53223,16209456731,"[хлеб, кефир]",116546,ш-д
53224,16209470290,"[напиток, пельмени]",109601,пакет


In [365]:
model = Word2Vec(
    train_grouped.category_noun.tolist(), vector_size=64, 
    sg=1, epochs=50, negative=10, min_count=1, 
    window=3, seed=42, workers=8
)

X = model.wv.key_to_index.keys()
len(X)

1799

In [378]:
model.save("supermarket_word2vec.model")

In [366]:
val_grouped['prediction'] = val_grouped['category_noun_x'].apply(lambda x: similar_products(aggregate_vectors(x), x))
val_grouped['predicted'] = val_grouped['prediction'].apply(lambda x: [xi[0] for xi in x])
val_grouped['accuracy_1'] = val_grouped[['category_noun_y', 'predicted']].apply(lambda x: 1 if len(x[1])>0 and x[0]==x[1][0] else 0, axis=1)
val_grouped['accuracy_2'] = val_grouped[['category_noun_y', 'predicted']].apply(lambda x: 1 if  len(x[1])>0 and x[0] in x[1] else 0, axis=1)

/opt/conda/lib/python3.10/site-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,


invalid index to scalar variable.
invalid index to scalar variable.
invalid index to scalar variable.
invalid index to scalar variable.


In [368]:
print("hit@5 valid: %.2f" % (val_grouped["accuracy_2"].mean() * 100))
print("accuracy valid: %.2f" % (val_grouped["accuracy_1"].mean() * 100))

hit@5 valid: 11.32
accuracy valid: 3.69


In [379]:
products = pd.concat([train[['receipt_id', 'item_id', 
       'name', 'price', 'quantity','category_noun']].drop_duplicates(),
                      val[['device_id', 'receipt_id', 'item_id', 
       'name', 'price', 'quantity','category_noun']].drop_duplicates()])

In [395]:
cosmetics = pd.concat([train[['receipt_id', 'item_id', 
       'name', 'price', 'quantity','category_noun']].drop_duplicates(),
                      val[['device_id', 'receipt_id', 'item_id', 
       'name', 'price', 'quantity','category_noun']].drop_duplicates()])

In [401]:
products['type']='super'
cosmetics['type']='cosmetic'

In [398]:
products.shape, cosmetics.shape

((840016, 6), (275556, 6))

In [402]:
pd.concat([products, cosmetics]).to_csv('file_for_db.csv', sep=';', index=None)